# Generating data from results
In order to compare the method of hypedsearch against other tools (i.e. NeoFusion) we should start with mock data. This data will be generated from real results. The purpose of this is to get a somewhat real distribution of proteins, peptide lengths, and other properties from real data. Results from SpectrumMill from Delong Lab will be used. The steps in doing this are as follows: 
1. Load the results from SpectrumMill
2. Generate .mzML files to run through everything (make sure to keep track of which peptides are hybrids and whatnot)

Information on the peptides (i.e. parent protein, is a hybrid, etc) is kept in a json

## 1. Convert SpectrumMill ssv results to experiment json

### Load ssv file into pandas df (because its easy)

In [1]:
import pandas as pd

results_file = '/Users/zacharymcgrath/Downloads/NOD2_E3_results.ssv'
df = pd.read_csv(results_file, ';')
df.head(2)

,number,filename,parent_charge,score,deltaForwardReverseScore,deltaRank1Rank2Score,percent_scored_peak_intensity,totalIntensity,previous_aa,sequence,next_aa,retentionTimeMin,chromatographicPeakWidthSec,parent_m_over_z,species,entry_name
0,1,NOD2_E3.13446.13477.2,2,10.10,10.10,9.91,84.5,183000.0,(E),DPQVEQLEL,(-),48.35,26.0,535.7725,MOUSE,ins1C18
1,2,NOD2_E3.18005.18246.2,2,12.84,11.07,12.84,97.8,40000000.0,(G),DLQTLALEVA,(-),65.78,29.0,536.8007,MOUSE,ins1C3


In [2]:
SEQUENCE_COL_NAME = 'sequence'
PARENT_PROTEIN_NAME = 'entry_name'

In [3]:
import sys
sys.path.append('/Users/zacharymcgrath/Documents/Layer_Research/Proteomics_Experiments/Database_Experiments/src')
from file_io import fasta

# the databse is shown in the column "species" so we'll just grab that manually from UniProt
database_file_name = '/Users/zacharymcgrath/Downloads/uniprot-proteome_UP000000589.fasta'
db_list = fasta.read(database_file_name, is_uniprot=True)
print(db_list[1])

{'name': 'SDHL_MOUSE', 'sequence': 'MAAQESLHVKTPLRDSMALSKLAGTSVFLKMDSSQPSGSFKIRGIGHLCKMKAKQGCRHFVCSSAGNAGMATAYAARRLGIPATIVVPNTTPALTIERLKNEGATVEVVGEMLDEAIQVAKALEKNNPGWVYISPFDDPLIWEGHTSLVKELKETLSAKPGAIVLSVGGGGLLCGVVQGLREVGWEDVPIIAMETFGAHSFHAAIKEGKLVTLPKITSVAKALGVNTVGAQTLKLFYEHPIFSEVISDQEAVSALEKFVDDEKILVEPACGAALAAVYSRVVCRLQDEGRLQTPLASLVVIVCGGSNISLAQLQALKVQLGLNGLPE', 'identifier': 'Q8VBT2', 'human_readable_name': 'L-serine dehydratase/L-threonine deaminase'}


#### switch list representation of database to dictionary based on the 'name' field

In [4]:
from collections import defaultdict
db_dict = defaultdict(list)

for e in db_list:
    db_dict[e['human_readable_name']].append({'sequence': e['sequence'], 'id': e['identifier'], 'name': e['name']})

In [5]:
db_dict['Peroxiredoxin-6']

[{'sequence': 'MPGGLLLGDEAPNFEANTTIGRIRFHDFLGDSWGILFSHPRDFTPVCTTELGRAAKLAPEFAKRNVKLIALSIDSVEDHLAWSKDINAYNGETPTEKLPFPIIDDKGRDLAILLGMLDPVEKDDNNMPVTARVVFIFGPDKKLKLSILYPATTGRNFDEILRVVDSLQLTGTKPVATPVDWKKGESVMVVPTLSEEEAKQCFPKGVFTKELPSGKKYLRYTPQP',
  'id': 'O08709',
  'name': 'PRDX6_MOUSE'},
 {'sequence': 'MKWHLKMRWGILFSHPRDFTPVCTTELGRAAKLAPEFAKRNVKLIALSIDSVEDHLAWSKDINAYNGETPTEKLPFPIIDDKGRDLAILLGMLDPVEKDANNMPVTARVVFIFGPDKKLKLSILYPATTGRNFDEILRVVDSLQLTGTKPVATPVDWKKGESVMVVPTLSEEEAKQCFPKGVFTKELPSGKKYLRYTPQP',
  'id': 'D3Z0Y2',
  'name': 'D3Z0Y2_MOUSE'},
 {'sequence': 'MPGGLLLGDEAPNFEANTTIGRIRFHDFLGDSWGILFSHPRDFTPVCTTELGRAAKLAPEFAKRNVKLIALSIDSVEDHLAWSKDINAYNGETPTEKLPFPIIDDKGRDLAILLGMLDPVEKDANNMPVTARVVFIFGPDKKLKLSILYPATTGRNFDEILRVVDSLQLTGTKPVATPVDWKKGESVMVVPTLSEEEAKQCFPKGVFTKELPSGKKYLRYTPQP',
  'id': 'Q6GT24',
  'name': 'Q6GT24_MOUSE'},
 {'sequence': 'MPGGLLLGDEAPNFEANTTIGRIRFHDFLGDSTSMLTMVKHPRKSCHFPSLMIRAGTLPSFWACWIQSRRTLTTCL',
  'id': 'A0A0A6YXQ7',
  'name': 'A0A0A6YXQ7_MOUSE'}]

### we now have all of our proteins indexable in db_hmn_dict by 'entry_name' in df

## Using the above info, create info part of the dictionary 
### Structure
The high level structure is
```python
{'experiment_info': {
    'proteins': [],
    'peptides': []
}}
```
Protein structure is 
```python
{
    "name": str,
    "human_readable_name": str,
    "sequence": str,
    "identifier": str
}
```
peptide structure is 
```python
{
    "peptide_name": "peptide_00",
    "peptide_sequence": "ALWGPDPAAAFVNQH",
    "parent_name": str,
    "parent_sequence": str",
    "starting_position": 14,
    "ending_position": 28 #inclusive
}
```

In [13]:
import regex as re
exp = {'experiment_info': {
    'proteins': [],
    'peptides': []
}}
start_len = len(df)
missing = 0
fill_zeroes = len(str(len(df)))

protein_count = defaultdict(int)
missing_dict ={}

def rm_parens(s):
    if '(' in s:
        print(s)
        print(re.sub(r'\([^)]*\)', '', s))
    return(re.sub(r'\([^)]*\)', '', s))

def get_parent_info(name, pepseq):
    if not isinstance(db_dict[name], list):
        return db_dict[name]
    else:
        for e in db_dict[name]:
            if pepseq in e['sequence']:
                return e
        
for idx, row in df.iterrows():
    if row[PARENT_PROTEIN_NAME] not in db_dict:
        missing += 1
        if row[PARENT_PROTEIN_NAME] not in missing_dict: 
            missing_dict[row[PARENT_PROTEIN_NAME]] = []        
        
        pepseq = rm_parens(row[SEQUENCE_COL_NAME])
        parenthmnname = row[PARENT_PROTEIN_NAME]
        missing_dict[row[PARENT_PROTEIN_NAME]].append({
            'peptide_sequence': pepseq,
            'parent_human_readable_name': parenthmnname,
        })
        continue
        
    pepname = 'peptide_'+ str(idx).zfill(fill_zeroes)
    pepseq = rm_parens(row[SEQUENCE_COL_NAME])
    parent_info = get_parent_info(row[PARENT_PROTEIN_NAME], pepseq)
    parentname = parent_info['name']
    parenthmnname = row[PARENT_PROTEIN_NAME]
    parentseq = parent_info['sequence']
    startpos = parentseq.index(pepseq)
    endpos = startpos + len(pepseq) - 1
    exp['experiment_info']['peptides'].append({
        'peptide_name': pepname,
        'peptide_sequence': pepseq,
        'parent_name': parentname,
        'parent_human_readable_name': parenthmnname,
        'parent_sequence': parentseq,
        'starting_position': startpos,
        'ending_position': endpos
    })
    protein_count[parentname] += 1
    
print('Missing {} of {} peptides due to parent names\n'.format(missing, start_len))
print(missing_dict)

Missing 12 of 1086 peptides due to parent names

{'ins1C18': [{'peptide_sequence': 'DPQVEQLEL', 'parent_human_readable_name': 'ins1C18'}], 'ins1C3': [{'peptide_sequence': 'DLQTLALEVA', 'parent_human_readable_name': 'ins1C3'}], 'ins1C5': [{'peptide_sequence': 'DLQTLALE', 'parent_human_readable_name': 'ins1C5'}], 'ins1C6': [{'peptide_sequence': 'DLQTLAL', 'parent_human_readable_name': 'ins1C6'}], 'HYBRID: mouse ins1C PQVEQLELGGSPGDLQTLAL-LLDEGHYPVRESPIDTAKRY mouse scg1': [{'peptide_sequence': 'DLQTLALLL', 'parent_human_readable_name': 'HYBRID: mouse ins1C PQVEQLELGGSPGDLQTLAL-LLDEGHYPVRESPIDTAKRY mouse scg1'}], 'HYBRID: mouse ins2C EVEDPQVAQLELGG-EVEDPQVAQLELGGGPGAGD mouse ins2': [{'peptide_sequence': 'DPQVAQLELGGEVEDPQVAQLELGGGPGAG', 'parent_human_readable_name': 'HYBRID: mouse ins2C EVEDPQVAQLELGG-EVEDPQVAQLELGGGPGAGD mouse ins2'}], 'HYBRID: mouse ins2C EVEDPQVAQLELGGGPGAGD-LPVNSPMTKGDTKVMKCVLE mouse chgA': [{'peptide_sequence': 'DLPVNSPMTKG', 'parent_human_readable_name': 'HYBRID: mou

In [7]:
from copy import deepcopy
for hmnrdblname in db_dict:
    e = deepcopy(db_dict[hmnrdblname])
    if isinstance(e, list):
        for el in e:
            if el['name'] not in protein_count:
                continue
            el['human_readable_name'] = hmnrdblname
            exp['experiment_info']['proteins'].append(el)
    else:
        if e['name'] not in protein_count: 
            continue
        e['human_readable_name'] = hmnrdblname
        exp['experiment_info']['proteins'].append(e)

In [8]:
print(len(exp['experiment_info']['proteins']))

280


## 2. Generate the .mzML file